In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import csv

driver = webdriver.Edge()
driver.get("https://shopee.vn/mall/brands/11035639")
driver.implicitly_wait(0.5)
driver.maximize_window()
driver.implicitly_wait(2)

for _ in range(20):
    driver.execute_script("window.scrollBy(0, 600)")
    driver.implicitly_wait(2)
    
list_full_brand = driver.find_elements(By.CLASS_NAME, "full-brand-list-item")   

list_shop_path = []
list_shop_name = []
for brand in list_full_brand:
    item_shop_name = brand.find_element(By.CLASS_NAME, "full-brand-list-item__brand-name").text
    item_shop_path = brand.find_element(By.TAG_NAME, 'a').get_attribute("href")
    
    list_shop_name.append(item_shop_name)
    list_shop_path.append(item_shop_path)
    
print("Total shop:", len(list_shop_path))

# Get each shop
for index_shop, shop_path in enumerate(list_shop_path):
    # Login if fist shop
    if index_shop == 0:
        driver.get(shop_path + "#product_list")
        
        text_box = driver.find_element(by=By.NAME, value="loginKey")
        text_box.send_keys("clonevector")
        sleep(1)
        text_box = driver.find_element(by=By.NAME, value="password")
        text_box.send_keys("Phuoc12345678")
        sleep(1)
        button = driver.find_element(By.CLASS_NAME, "_1EApiB")
        sleep(1)
        button.click()
    
    # Continue crawl if error
    if index_shop < 141:
        continue    
    
    print("index_shop", index_shop, "shop_path", shop_path)
    driver.get(shop_path + "#product_list")
    
    sleep(3)
    
    previous_index = 0
    current_index = 0
    while True:
        current_index = driver.find_element(By.CLASS_NAME, "shopee-button-solid--primary").text

        if current_index == previous_index:
            break
        
        # Get full items in current index        
        list_items = driver.find_elements(By.CLASS_NAME, "shop-search-result-view__item")
        print("Total items:", len(list_items))
        
        results = []
        for item in list_items:
            item_data = []
            
            item_path = item.find_element(By.TAG_NAME, 'a').get_attribute("href")
            print("item_path:", item_path)
            if item_path == None:
                continue
            item_data.append(item_path)
            
            try:
                item_image = item.find_element(By.CLASS_NAME, 'vYyqCY').get_attribute("src")
                print("item_image:", item_image)
                if item_image == None:
                    continue
                item_data.append(item_image)
            except:
                print("error item_image")
                continue
            
            item_name = item.find_element(By.CLASS_NAME, "h0HBrE").text
            print("item_name:", item_name)
            if item_name == None:
                continue
            item_data.append(item_name)

            fixed_item_price = item.find_element(By.CLASS_NAME, "zSpiUB").text
            print("fixed_item_price:", fixed_item_price)
            if fixed_item_price == None:
                continue
            item_data.append(fixed_item_price)
            
            try:
                sale_item_price = item.find_element(By.CLASS_NAME, "_0ZJOIv").text
            except:
                sale_item_price = None
            print("sale_item_price:", sale_item_price)
            item_data.append(sale_item_price)            
            
            try:
                sales_number = item.find_element(By.CLASS_NAME, "sPnnFI").text
                if sales_number == None:
                    sales_number = "Đã bán 0"
            except:
                sales_number = "Đã bán 0"
            print("sales_number:", sales_number)
            item_data.append(sales_number)     
            
            print("shop_path:", shop_path)
            item_data.append(shop_path)
            
            shop_name = list_shop_name[index_shop]
            print("shop_name:", shop_name)
            item_data.append(shop_name)
            
            results.append(item_data)
            
            print("--------------------------------------------")
        
        # Write data to file csv
        with open(f"./data.csv", "a", encoding='utf-8', newline='') as file:
            # Create a CSV writer
            writer = csv.writer(file)
            
            # Write data
            writer.writerows(results)

        
        # Next page
        button_next = driver.find_element(By.CLASS_NAME, "shopee-icon-button--right ")
        button_next.click()
        
        previous_index = current_index

        print("Shop--------------------------------------------End")
        
driver.close()